# Template Network Definitions

<i>Version 1</i>

## References

1. ["Maintaining Knowledge about Temporal Intervals" by James F. Allen](https://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf) - Allen's original paper (PDF)
1. [Allen's Interval Algebra](https://www.ics.uci.edu/~alspaugh/cls/shr/allen.html) or [here](https://thomasalspaugh.org/pub/fnd/allen.html) - summarizes Allen's algebra of proper time intervals
1. [W3C Time Ontology in OWL](https://www.w3.org/TR/owl-time/) - temporal vocabulary used here is based on the W3C vocabulary of time
1. [bitsets Python package](https://bitsets.readthedocs.io/en/stable/) - used to implement Algebra relation sets and operations
1. [NetworkX Python package](http://networkx.github.io/) - used to represent directed graph of constraints
1. [Python format string syntax](https://docs.python.org/3/library/string.html#format-string-syntax) - used in Algebra summary method
1. [Spatial Ontology](https://www.w3.org/2017/sdwig/bp/) - I'm still looking for a standard spatial vocabulary; maybe start here
1. [Qualitative Spatial Relations (QSR) Library](https://qsrlib.readthedocs.io/en/latest/index.html) - an alternative library to the one defined here

## Dependencies

In [23]:
import os
import qualreas as qr

In [24]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

In [41]:
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/LinearPointAlgebra.json"))
pt_alg = qr.Algebra(os.path.join(path, "Algebras/RightBranchingPointAlgebra.json"))
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/LeftBranchingPointAlgebra.json"))

## Create Template 4-Point Network

In [42]:
def four_point_network(alg, lessthan_symbol, verbose=False):
    net = qr.Network(alg, "Four Point Network")
    lessthan = alg.relset(lessthan_symbol)
    # Start & End Points of Interval 1
    start1 = qr.TemporalEntity(["Point"], name="StartPt1")
    end1   = qr.TemporalEntity(["Point"], name=  "EndPt1")
    # Start & End Points of Interval 2
    start2 = qr.TemporalEntity(["Point"], name="StartPt2")
    end2   = qr.TemporalEntity(["Point"], name=  "EndPt2")
    net.add_constraint(start1, end1, lessthan, verbose)
    net.add_constraint(start2, end2, lessthan, verbose)
    return net, start1, end1, start2, end2

In [43]:
net4pt, s1, e1, s2, e2 = four_point_network(pt_alg, "<|=")

In [44]:
net4pt.summary()


Four Point Network: 4 nodes, 8 edges
  Algebra: RightBranchingPointAlgebra
  StartPt1:
    => StartPt1: =
    => EndPt1: <|=
  EndPt1:
    => EndPt1: =
    => StartPt1: =|>
  StartPt2:
    => StartPt2: =
    => EndPt2: <|=
  EndPt2:
    => EndPt2: =
    => StartPt2: =|>


In [45]:
key_name_mapping = {
    '<,<,<,<': 'B', '>,>,>,>': 'BI',
    '>,<,>,<': 'D', '<,<,>,>': 'DI',
    '=,<,>,=': 'E', '=,=,=,=': 'PE',
    '>,<,>,=': 'F', '<,<,>,=': 'FI',
    '<,<,=,<': 'M', '>,=,>,>': 'MI',
    '<,<,>,<': 'O', '>,<,>,>': 'OI',
    '=,<,>,<': 'S', '=,<,>,>': 'SI',
    '>,=,>,=': 'PF', '<,<,=,=': 'PFI',
    '=,<,=,<': 'PS', '=,=,>,>': 'PSI'
}

In [46]:
def print_4point_constraint_matrix(net, s1, e1, s2, e2):
    pts = [s1, e1, s2, e2]
    for a in pts:
        row = ""
        for b in pts:
            row += "  " + str(net.edges[a, b]['constraint'])
        print(row)    

Viewing the network as a matrix, 'elem13', refers to the element in row 1, col 3, and so on for 'elem23', etc.  The matrix is 4x4, so if we partition it into four 2x2 matrices, then the two partiions on the diagonal represent two intervals and the two off-diagonal partitions represent how those two intervals relate to each other. Also, the off-diagonal 2x2 partitions are converse transposes of each other.  Oh, and the intervals represented by the diagonal partitions could be intervals, proper intervals, or points.

In [47]:
count = 0
consistent_nets = dict()
for elem13 in pt_alg.elements:
    for elem23 in pt_alg.elements:
        for elem14 in pt_alg.elements:
            for elem24 in pt_alg.elements:
                net, pt1, pt2, pt3, pt4 = four_point_network(pt_alg, "<|=")
                rs13 = pt_alg.relset(elem13)
                rs23 = pt_alg.relset(elem23)
                rs14 = pt_alg.relset(elem14)
                rs24 = pt_alg.relset(elem24)
                net.add_constraint(pt1, pt3, rs13)
                net.add_constraint(pt2, pt3, rs23)
                net.add_constraint(pt1, pt4, rs14)
                net.add_constraint(pt2, pt4, rs24)
                if net.propagate():
                    count += 1
                    elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                    print("======================")
                    if elem_key in key_name_mapping:
                        print(key_name_mapping[elem_key])
                    else:
                        print("UNKNOWN")
                    print_4point_constraint_matrix(net, pt1, pt2, pt3, pt4)
                    consistent_nets[elem_key] = net
print(f"\n{count} consistent networks")

B
  =  <|=  <  <
  =|>  =  <  <
  >  >  =  <|=
  >  >  =|>  =
M
  =  <  <  <
  >  =  =  <
  >  =  =  <
  >  >  >  =
PFI
  =  <  <  <
  >  =  =  =
  >  =  =  =
  >  =  =  =
O
  =  <  <  <
  >  =  >  <
  >  <  =  <
  >  >  >  =
FI
  =  <  <  <
  >  =  >  =
  >  <  =  <
  >  =  >  =
DI
  =  <  <  <
  >  =  >  >
  >  <  =  <|=
  >  <  =|>  =
UNKNOWN
  =  <  <  <
  >  =  >  r~
  >  <  =  <
  >  r~  >  =
UNKNOWN
  =  <  <  <
  >  =  r~  r~
  >  r~  =  <|=
  >  r~  =|>  =
PS
  =  =  =  <
  =  =  =  <
  =  =  =  <
  >  >  >  =
PE
  =  =  =  =
  =  =  =  =
  =  =  =  =
  =  =  =  =
S
  =  <  =  <
  >  =  >  <
  =  <  =  <
  >  >  >  =
E
  =  <  =  <
  >  =  >  =
  =  <  =  <
  >  =  >  =
SI
  =  <  =  <
  >  =  >  >
  =  <  =  <
  >  <  >  =
UNKNOWN
  =  <  =  <
  >  =  >  r~
  =  <  =  <
  >  r~  >  =
PSI
  =  <  =  =
  >  =  >  >
  =  <  =  =
  =  <  =  =
D
  =  <|=  >  <
  =|>  =  >  <
  <  <  =  <
  >  >  >  =
F
  =  <  >  <
  >  =  >  =
  <  <  =  <
  >  =  >  =
OI
  =  <  >  <
  >  =  >  

In [48]:
for key in consistent_nets:
    print(f"{key_name_mapping[key]} : {key} : {repr(consistent_nets[key])}")

B : <,<,<,< : <qualreas.Network object at 0x7fc938ac0850>
M : <,<,=,< : <qualreas.Network object at 0x7fc938ac0490>
PFI : <,<,=,= : <qualreas.Network object at 0x7fc93843cc50>
O : <,<,>,< : <qualreas.Network object at 0x7fc938ac0810>
FI : <,<,>,= : <qualreas.Network object at 0x7fc938431510>
DI : <,<,>,> : <qualreas.Network object at 0x7fc93843c6d0>


KeyError: '<,<,>,r~'